DAATA INSTALL
```
# 1. Install the library
!pip install SoccerNet

# 2. Download directly to Google Drive
from SoccerNet.Downloader import SoccerNetDownloader
import os

# Define the path to your Drive
# This creates a folder "SoccerNetData" in your main Drive folder
drive_path = "/content/drive/MyDrive/SoccerNetData"

if not os.path.exists(drive_path):
    os.makedirs(drive_path)

print(f"🚀 Downloading to: {drive_path}")

# Initialize Downloader with the Drive path
my_downloader = SoccerNetDownloader(LocalDirectory=drive_path)

# Download Tracking Data (Videos + Labels)
# Note: This will take time, but it runs on Google's servers, not yours!
my_downloader.downloadDataTask(task="tracking", split=["train"])

# This is formatted as code
```



In [ ]:
!pip install ultralytics opencv-python-headless -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics opencv-python tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.2 MB/s eta 0:00:00


In [ ]:
import os
import json
import glob
import shutil
import random
import cv2
import pandas as pd
from tqdm import tqdm
from ultralytics import YOLO
from google.colab import drive

# ==============================================================================
# 1. SETUP & MOUNT (Force Remount to fix any timeouts)
# ==============================================================================
print("🔄 Setting up Google Drive...")
try:
    drive.flush_and_unmount()
except:
    pass
drive.mount('/content/drive', force_remount=True)

# ==============================================================================
# 🔧 PATH CONFIGURATION
# ==============================================================================
# INPUT: Raw Data on Google Drive
ORGANIZED_ROOT = "/content/drive/MyDrive/SoccerNetData/tracking/train_extracted/train/organized"
SNMOT_ROOT = "/content/drive/MyDrive/SoccerNetData/tracking/train_extracted/train"

# OUTPUT: Fast Local SSD (We build the dataset here directly)
DATASET_DIR = "/content/soccer_dataset"

# ==============================================================================
# 🛠️ HELPER FUNCTIONS
# ==============================================================================
def get_yolo_class(role, csv_cls):
    # 0=Ball, 1=Player, 2=GK, 3=Ref
    role = role.lower() if role else ""
    if role == "ball" or csv_cls == 4: return 0
    if role == "goalkeeper": return 2
    if role == "referee" or csv_cls == 3: return 3
    return 1 # Default to Player

def convert_bbox(img_w, img_h, x, y, w, h):
    # Convert to normalized xywh
    cx = (x + w/2) / img_w
    cy = (y + h/2) / img_h
    nw = w / img_w
    nh = h / img_h
    return max(0, min(1, cx)), max(0, min(1, cy)), max(0, min(1, nw)), max(0, min(1, nh))

# ==============================================================================
# 🏗️ BUILD DATASET (Drive -> Local SSD)
# ==============================================================================
def build_dataset_full():
    if os.path.exists(DATASET_DIR):
        print(f"🧹 Cleaning up old local data...")
        shutil.rmtree(DATASET_DIR)

    # Create directories
    for s in ["train", "val"]:
        os.makedirs(f"{DATASET_DIR}/images/{s}", exist_ok=True)
        os.makedirs(f"{DATASET_DIR}/labels/{s}", exist_ok=True)

    print(f"🔍 Scanning Drive for all matches...")
    # Find all JSON files (one per match clip)
    json_files = glob.glob(os.path.join(ORGANIZED_ROOT, "**", "Labels-GameState.json"), recursive=True)

    if not json_files:
        print("❌ ERROR: No Label files found! Check your Drive paths.")
        return False

    print(f"✅ Found {len(json_files)} clips. Processing full dataset...")
    print("☕ This conversion will take 15-20 mins. It is building the data on the A100 SSD.")

    total_images = 0

    # Process every single clip
    for json_path in tqdm(json_files, desc="Converting Clips"):
        try:
            # 1. Identify Clip ID (e.g., SNGS-050 -> 050)
            sngs_folder = os.path.basename(os.path.dirname(json_path))
            clip_id = sngs_folder.split("-")[-1]

            # 2. Locate Images & GT (Handle potential missing folders)
            snmot_folder = os.path.join(SNMOT_ROOT, f"SNMOT-{clip_id}")
            img_dir = os.path.join(snmot_folder, "img1")
            gt_path = os.path.join(snmot_folder, "gt", "gt.txt")

            if not os.path.exists(img_dir) or not os.path.exists(gt_path):
                continue # Skip if raw data is missing

            # 3. Load Roles (JSON)
            with open(json_path) as f:
                data = json.load(f)
            role_map = {}
            for ann in data.get("annotations", []):
                tid = ann.get("track_id")
                role = ann.get("attributes", {}).get("role")
                if tid and role:
                    role_map[int(tid)] = role

            # 4. Load BBoxes (CSV)
            # frame, id, x, y, w, h, conf, class, visibility, _
            df = pd.read_csv(gt_path, header=None)
            df.columns = ['frame', 'id', 'x', 'y', 'w', 'h', 'c', 'cls', 'v', 'r']

            # 5. Get Dimensions (Once per clip)
            images = sorted([f for f in os.listdir(img_dir) if f.endswith(".jpg")])
            if not images: continue

            # Read one image to get size (Fastest method)
            sample_img = cv2.imread(os.path.join(img_dir, images[0]))
            if sample_img is None: continue
            h_img, w_img = sample_img.shape[:2]

            # 6. Process Images
            for img_name in images:
                frame_idx = int(img_name.split(".")[0])

                rows = df[df['frame'] == frame_idx]
                if rows.empty: continue

                # 90% Train / 10% Val split
                split = "train" if random.random() < 0.9 else "val"

                # Unique name to prevent overwrites
                new_name = f"{clip_id}_{img_name}"

                # Destination Paths (Local SSD)
                src_img = os.path.join(img_dir, img_name)
                dst_img = os.path.join(DATASET_DIR, "images", split, new_name)
                dst_lbl = os.path.join(DATASET_DIR, "labels", split, new_name.replace(".jpg", ".txt"))

                # Generate Label Content
                label_data = []
                for _, row in rows.iterrows():
                    tid = int(row['id'])
                    role = role_map.get(tid, "player")
                    cls_idx = get_yolo_class(role, int(row['cls']))
                    cx, cy, nw, nh = convert_bbox(w_img, h_img, row['x'], row['y'], row['w'], row['h'])
                    label_data.append(f"{cls_idx} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}\n")

                # Only write if we have labels
                if label_data:
                    with open(dst_lbl, "w") as f_out:
                        f_out.writelines(label_data)
                    shutil.copy(src_img, dst_img) # Copy image to Local SSD
                    total_images += 1

        except Exception as e:
            # Print error but keep going
            print(f"⚠️ Warning in {sngs_folder}: {e}")
            continue

    # Create YAML Config
    yaml_content = f"""
path: {DATASET_DIR}
train: images/train
val: images/val

nc: 4
names:
  0: ball
  1: player
  2: goalkeeper
  3: referee
"""
    with open(f"{DATASET_DIR}/data.yaml", "w") as f:
        f.write(yaml_content)

    print(f"\n✅ DATASET READY: {total_images} images prepared on Local SSD.")
    return True

# ==============================================================================
# 🚀 TRAIN (A100 POWER)
# ==============================================================================
if build_dataset_full():
    print("🔥 Starting Full Training on A100 GPU...")

    model = YOLO("yolov8m.pt")

    model.train(
        data=f"{DATASET_DIR}/data.yaml",
        epochs=20,
        imgsz=960,

        # A100 OPTIMIZED SETTINGS
        batch=24,        # 24 is safe. You can try 32 if you want to push it.
        workers=16,      # High workers because data is on local SSD
        device=0,

        project="/content/drive/MyDrive/SoccerTraining", # Save results to Drive
        name="soccer_full_A100",
        exist_ok=True
    )

    print("🎉 Training Complete! Model saved to Google Drive.")
else:
    print("❌ Dataset build failed.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🔄 Setting up Google Drive...
Mounted at /content/drive
🔍 Scanning Drive for all matches...
✅ Found 57 clips. Processing full dataset...
☕ This conversion will take 15-20 mins. It is building the data on the A100 SSD.


Converting Clips: 100%|██████████| 57/57 [1:44:52<00:00, 110.40s/it]



✅ DATASET READY: 42750 images prepared on Local SSD.
🔥 Starting Full Training on A100 GPU...
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40441MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/soccer_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=soccer_f